# Data preparation

In [3]:
import pandas as pd

### Load dbs

In [66]:
pubchem_db = r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\original_dbs\pubchem_human_5ht6.csv'
chembl_db = r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\original_dbs\chembl_human_5ht6.csv'

In [243]:
pubchem_df = pd.read_csv(pubchem_db, sep=',')
chembl_df = pd.read_csv(chembl_db, sep=';')

### Filter out unnecessary columns

In [12]:
pubchem_df.columns

Index(['baid', 'activity', 'aid', 'sid', 'cid', 'geneid', 'pmid', 'aidtype',
       'aidmdate', 'hasdrc', 'rnai', 'protacxn', 'acname', 'acqualifier',
       'acvalue', 'aidsrcname', 'aidname', 'cmpdname', 'targetname',
       'targeturl', 'ecs', 'repacxn', 'taxids', 'cellids', 'targettaxid'],
      dtype='object')

In [14]:
chembl_df.columns

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties'],
      dtype='object')

In [155]:
pubchem_columns = ['sid', 'acname', 'acqualifier', 'acvalue', 'activity']
chembl_columns = ['Smiles', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units', 'pChEMBL Value']

In [244]:
pubchem_df = pubchem_df.drop(columns=[column for column in pubchem_df.columns if column not in pubchem_columns])
chembl_df = chembl_df.drop(columns=[column for column in chembl_df.columns if column not in chembl_columns])

### Add smiles to pubchem dataset

Smiles were obtained through Pubchem Identifier Exchange Service based on SID

In [157]:
pubchem_smiles = r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\original_dbs\pubchem_db_smiles.csv'
pubchem_smiles_df = pd.read_csv(pubchem_smiles, sep="\t")

In [245]:
pubchem_df = pd.merge(pubchem_df, pubchem_smiles_df, how='left')

In [246]:
pubchem_df.drop_duplicates(subset=['sid'], inplace=True)

In [247]:
# renaming columns so that they much ChEMBL dataset column names
pubchem_df = pubchem_df.rename(columns= {'acname': 'Standard Type', 'acqualifier': 'Standard Relation', 'acvalue': 'Standard Value'})

### Clean pubchem dataset

In [248]:
pubchem_df = pubchem_df.drop(pubchem_df[pubchem_df['Standard Type'] != 'Ki'].index)
pubchem_df = pubchem_df.drop(pubchem_df[pubchem_df['Standard Relation'] != '='].index)
pubchem_df = pubchem_df.drop(pubchem_df[pubchem_df['activity'] != 'Active'].index)
pubchem_df = pubchem_df.drop(columns=['activity'])
pubchem_df.dropna(inplace=True)

In [249]:
# Convert mikro mol Ki to nmol
pubchem_df['Standard Value'] = pubchem_df['Standard Value'].multiply(1000)

In [250]:
pubchem_df = pubchem_df.drop(columns=['Standard Relation', 'sid'])

In [251]:
pubchem_df

,Standard Type,Standard Value,Smiles
2,Ki,1.0000,C1CCN(C1)C2=NC(=CC(=C2)S(=O)(=O)C3=CC=C(C=C3)N)Br
8,Ki,2.2910,CNC1=NC(=CC(=C1)S(=O)(=O)C2=CC=C(C=C2)N)Br
9,Ki,0.1148,C1CN(CCN1)C2=NC(=CC(=C2)S(=O)(=O)C3=CC=C(C=C3)...
11,Ki,53.7000,C1=CC(=CC=C1N)S(=O)(=O)C2=CC(=NC(=C2)Br)Br
13,Ki,54.9500,CNC1=CC(=NC(=N1)NC)NS(=O)(=O)C2=CC=C(C=C2)N.Br.Br
...,...,...,...
18349,Ki,102.0000,C1CNCC=C1C2=CN(C3=C2C=CC(=C3)Cl)S(=O)(=O)C4=CC...
18350,Ki,30.0000,COC1=CC2=C(C=C1)N(C=C2C3CCNCC3)S(=O)(=O)C4=CC5...
18351,Ki,14.0000,COC1=CC2=C(C=C1)N(C=C2C3=CCNCC3)S(=O)(=O)C4=CC...
18352,Ki,7.0000,C1CNCC=C1C2=CN(C3=CC=CC=C32)S(=O)(=O)C4=CC5=C(...


In [255]:
pubchem_df.to_csv(r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\clean_dbs\pubchem_human_5ht6.csv', sep='\t')

### Clean chembl dataset

In [252]:
chembl_df = chembl_df.drop(chembl_df[chembl_df['Standard Type'] != 'Ki'].index)
chembl_df = chembl_df.drop(chembl_df[chembl_df['Standard Relation'] != "'='"].index)
chembl_df.drop_duplicates(subset=['Smiles'], inplace=True)
chembl_df.dropna(inplace=True)

In [253]:
chembl_df = chembl_df.drop(columns=['Standard Relation', 'Standard Units'])

In [254]:
chembl_df

,Smiles,Standard Type,Standard Value,pChEMBL Value
0,CC(C)c1ccc(S(=O)(=O)n2cc(N3CCN(C)CC3)c3ccccc32...,Ki,3.400,8.47
1,COc1ccc(Cl)cc1NS(=O)(=O)c1ccc(OC)c(N2CCNCC2)c1,Ki,1.259,8.90
2,CNc1cc(S(=O)(=O)Nc2ccc(N)cc2)cc(Br)n1,Ki,42.660,7.37
4,CC([Se]c1ccccc1)c1nc(N)nc(N2CCN(C)CC2)n1,Ki,111.000,6.96
6,CN1CCN(c2nc(N)nc(CSc3ccccc3)n2)CC1,Ki,26.000,7.58
...,...,...,...,...
5193,COc1ccc(Cc2nc(N3CCNCC3)nc3cccnc23)c(OC)c1,Ki,58.000,7.24
5201,O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1,Ki,20.000,7.70
5202,O=S(=O)(c1cccc2ncccc12)N1CCCCC[C@H]1CCN1CCN(c2...,Ki,176.000,6.75
5203,O=S(=O)(c1cncc2ccccc12)N1CC[C@@H]1CCN1CCN(c2cc...,Ki,104.000,6.98


In [264]:
chembl_df.to_csv(r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\clean_dbs\chembl_human_5ht6.csv', sep='\t')

### Concatenate the two dataframes

In [257]:
dataframes = [pubchem_df, chembl_df]
concat_df = pd.concat(dataframes)

In [258]:
concat_df

,Standard Type,Standard Value,Smiles,pChEMBL Value
2,Ki,1.0000,C1CCN(C1)C2=NC(=CC(=C2)S(=O)(=O)C3=CC=C(C=C3)N)Br,NaN
8,Ki,2.2910,CNC1=NC(=CC(=C1)S(=O)(=O)C2=CC=C(C=C2)N)Br,NaN
9,Ki,0.1148,C1CN(CCN1)C2=NC(=CC(=C2)S(=O)(=O)C3=CC=C(C=C3)...,NaN
11,Ki,53.7000,C1=CC(=CC=C1N)S(=O)(=O)C2=CC(=NC(=C2)Br)Br,NaN
13,Ki,54.9500,CNC1=CC(=NC(=N1)NC)NS(=O)(=O)C2=CC=C(C=C2)N.Br.Br,NaN
...,...,...,...,...
5193,Ki,58.0000,COc1ccc(Cc2nc(N3CCNCC3)nc3cccnc23)c(OC)c1,7.24
5201,Ki,20.0000,O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1,7.70
5202,Ki,176.0000,O=S(=O)(c1cccc2ncccc12)N1CCCCC[C@H]1CCN1CCN(c2...,6.75
5203,Ki,104.0000,O=S(=O)(c1cncc2ccccc12)N1CC[C@@H]1CCN1CCN(c2cc...,6.98


In [259]:
concat_df.to_csv(r'C:\Users\pecho\Documents\serotonin_receptor_project\databases\clean_dbs\concat_db_human_5ht6.csv', sep='\t')